In [1]:
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import time
import matplotlib.pyplot as plt

In [5]:
BATCH_SIZE = 32
EPOCH = 8
LR = 0.0001
new_model_train = True
device = "cuda" if torch.cuda.is_available() else "cpu"
criterion = nn.CrossEntropyLoss()
model_type = "PreTrained_ResNet50_SavemodelTest"
dataset = "dataset"
save_model_path = f"result/{model_type}{dataset}.pt"
save_history_path = f"result/{model_type}history{dataset}.pt"

In [3]:
device

'cpu'

In [4]:
train_dir = 'dataset/train'
valid_dir = 'dataset/valid'
test_dir = 'dataset/test'

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_DS = datasets.ImageFolder(train_dir, transform=train_transform)
valid_DS = datasets.ImageFolder(valid_dir, transform=valid_transform)
test_DS = datasets.ImageFolder(test_dir, transform=test_transform)

train_DL = DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
valid_DL = DataLoader(valid_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=True)

NameError: name 'BATCH_SIZE' is not defined